In [2]:
import requests
import pandas as pd
from tqdm import tqdm_notebook
import dask
import dask.bag as db
from dask.diagnostics import ProgressBar
URL = 'https://api.github.com/repos/matplotlib/matplotlib'

In [3]:
contrib = []
for p, _ in enumerate(tqdm_notebook(range(0,828,100))):
    r = requests.get(f'{URL}/contributors', params={'page':p+1, 'per_page':100, 'anon':True})
    contrib.extend(r.json())

In [4]:
cdf = pd.io.json.json_normalize(contrib)

In [5]:
cdf.head()

,login,id,node_id,avatar_url,gravatar_id,url,html_url,followers_url,following_url,gists_url,...,subscriptions_url,organizations_url,repos_url,events_url,received_events_url,type,site_admin,contributions,email,name
0,tacaswell,199813.0,MDQ6VXNlcjE5OTgxMw==,https://avatars3.githubusercontent.com/u/19981...,,https://api.github.com/users/tacaswell,https://github.com/tacaswell,https://api.github.com/users/tacaswell/followers,https://api.github.com/users/tacaswell/followi...,https://api.github.com/users/tacaswell/gists{/...,...,https://api.github.com/users/tacaswell/subscri...,https://api.github.com/users/tacaswell/orgs,https://api.github.com/users/tacaswell/repos,https://api.github.com/users/tacaswell/events{...,https://api.github.com/users/tacaswell/receive...,User,False,4463,NaN,NaN
1,mdboom,38294.0,MDQ6VXNlcjM4Mjk0,https://avatars3.githubusercontent.com/u/38294...,,https://api.github.com/users/mdboom,https://github.com/mdboom,https://api.github.com/users/mdboom/followers,https://api.github.com/users/mdboom/following{...,https://api.github.com/users/mdboom/gists{/gis...,...,https://api.github.com/users/mdboom/subscriptions,https://api.github.com/users/mdboom/orgs,https://api.github.com/users/mdboom/repos,https://api.github.com/users/mdboom/events{/pr...,https://api.github.com/users/mdboom/received_e...,User,False,3918,NaN,NaN
2,jdh2358,38334.0,MDQ6VXNlcjM4MzM0,https://avatars1.githubusercontent.com/u/38334...,,https://api.github.com/users/jdh2358,https://github.com/jdh2358,https://api.github.com/users/jdh2358/followers,https://api.github.com/users/jdh2358/following...,https://api.github.com/users/jdh2358/gists{/gi...,...,https://api.github.com/users/jdh2358/subscript...,https://api.github.com/users/jdh2358/orgs,https://api.github.com/users/jdh2358/repos,https://api.github.com/users/jdh2358/events{/p...,https://api.github.com/users/jdh2358/received_...,User,False,2145,NaN,NaN
3,anntzer,1322974.0,MDQ6VXNlcjEzMjI5NzQ=,https://avatars0.githubusercontent.com/u/13229...,,https://api.github.com/users/anntzer,https://github.com/anntzer,https://api.github.com/users/anntzer/followers,https://api.github.com/users/anntzer/following...,https://api.github.com/users/anntzer/gists{/gi...,...,https://api.github.com/users/anntzer/subscript...,https://api.github.com/users/anntzer/orgs,https://api.github.com/users/anntzer/repos,https://api.github.com/users/anntzer/events{/p...,https://api.github.com/users/anntzer/received_...,User,False,1929,NaN,NaN
4,efiring,85125.0,MDQ6VXNlcjg1MTI1,https://avatars1.githubusercontent.com/u/85125...,,https://api.github.com/users/efiring,https://github.com/efiring,https://api.github.com/users/efiring/followers,https://api.github.com/users/efiring/following...,https://api.github.com/users/efiring/gists{/gi...,...,https://api.github.com/users/efiring/subscript...,https://api.github.com/users/efiring/orgs,https://api.github.com/users/efiring/repos,https://api.github.com/users/efiring/events{/p...,https://api.github.com/users/efiring/received_...,User,False,1852,NaN,NaN


In [11]:
cdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 21 columns):
login                  431 non-null object
id                     431 non-null float64
node_id                431 non-null object
avatar_url             431 non-null object
gravatar_id            431 non-null object
url                    431 non-null object
html_url               431 non-null object
followers_url          431 non-null object
following_url          431 non-null object
gists_url              431 non-null object
starred_url            431 non-null object
subscriptions_url      431 non-null object
organizations_url      431 non-null object
repos_url              431 non-null object
events_url             431 non-null object
received_events_url    431 non-null object
type                   900 non-null object
site_admin             431 non-null object
contributions          900 non-null int64
email                  469 non-null object
name                   469 non-null

In [12]:
cdf.to_csv("contributors.csv", index=False)

In [48]:
#auth to not trigger abuse 
auth = ('user', 'password')

In [34]:
def paged_requests(page):
    r = requests.get(f'{URL}/pulls', auth=auth,
                     params={'page':page, 'per_page':100, 'state':'all'})
    return r.json()

In [35]:
pull_pages = db.from_sequence(range(1, (10000//100)))
pulls = pull_pages.map(paged_requests)
pull_dfs = pulls.map(pd.io.json.json_normalize)

In [36]:
with ProgressBar():
    #single threaded cause github abuse and yes, should just use loop at this point
     pull_requests = pull_dfs.compute(scheduler='single-threaded')

[########################################] | 100% Completed |  5min 30.8s


In [37]:
pdf = pd.concat(pull_requests, sort=False)

In [38]:
pdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9050 entries, 0 to 49
Columns: 335 entries, url to head.user
dtypes: bool(13), float64(20), int64(14), object(288)
memory usage: 22.4+ MB


In [39]:
#check that no abuse message
pdf.iloc[-1]

url                          https://api.github.com/repos/matplotlib/matplo...
id                                                                       76738
node_id                                           MDExOlB1bGxSZXF1ZXN0NzY3Mzg=
html_url                       https://github.com/matplotlib/matplotlib/pull/1
diff_url                     https://github.com/matplotlib/matplotlib/pull/...
                                                   ...                        
head.repo.license.name                                                   Other
head.repo.license.spdx_id                                          NOASSERTION
head.repo.license.url                                                      NaN
head.repo.license.node_id                                     MDc6TGljZW5zZTA=
head.user                                                                  NaN
Name: 49, Length: 335, dtype: object

In [40]:
pdf.to_csv("pull_requests.csv", index=False)

In [41]:
def paged_issue_requests(page):
    r = requests.get(f'{URL}/issues', auth=auth,
                     params={'page':page, 'per_page':100, 'state':'all'})
    return r.json()

In [49]:
#back to loop to not trigger abuse
issues = []
for p, _ in enumerate(tqdm_notebook(range(1, (6000//100)))):
    issues.append(paged_issue_requests(p))

In [50]:
issues[-1] #check for abuse trigger

[{'url': 'https://api.github.com/repos/matplotlib/matplotlib/issues/9166',
  'repository_url': 'https://api.github.com/repos/matplotlib/matplotlib',
  'labels_url': 'https://api.github.com/repos/matplotlib/matplotlib/issues/9166/labels{/name}',
  'comments_url': 'https://api.github.com/repos/matplotlib/matplotlib/issues/9166/comments',
  'events_url': 'https://api.github.com/repos/matplotlib/matplotlib/issues/9166/events',
  'html_url': 'https://github.com/matplotlib/matplotlib/pull/9166',
  'id': 256023986,
  'node_id': 'MDExOlB1bGxSZXF1ZXN0MTM5ODcwNTQz',
  'number': 9166,
  'title': 'rst markup simplification',
  'user': {'login': 'anntzer',
   'id': 1322974,
   'node_id': 'MDQ6VXNlcjEzMjI5NzQ=',
   'avatar_url': 'https://avatars0.githubusercontent.com/u/1322974?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/anntzer',
   'html_url': 'https://github.com/anntzer',
   'followers_url': 'https://api.github.com/users/anntzer/followers',
   'following_url': 'https://api

In [52]:
#bit this bit can still be parallized
issues_dfs = db.from_sequence(issues).map(pd.io.json.json_normalize)

In [53]:
with ProgressBar():
    issues_all = issues_dfs.compute()

[########################################] | 100% Completed |  8.9s


In [54]:
issues_all[-1]

,url,repository_url,labels_url,comments_url,events_url,html_url,id,node_id,number,title,...,milestone.creator.received_events_url,milestone.creator.type,milestone.creator.site_admin,milestone.open_issues,milestone.closed_issues,milestone.state,milestone.created_at,milestone.updated_at,milestone.due_on,milestone.closed_at
0,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://github.com/matplotlib/matplotlib/pull/...,256023986,MDExOlB1bGxSZXF1ZXN0MTM5ODcwNTQz,9166,rst markup simplification,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://github.com/matplotlib/matplotlib/issue...,256019466,MDU6SXNzdWUyNTYwMTk0NjY=,9165,Public API to modify mathtext font constants,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://github.com/matplotlib/matplotlib/pull/...,255781472,MDExOlB1bGxSZXF1ZXN0MTM5NjkzODkz,9164,include overspilling axes legends in ax.get_ti...,...,https://api.github.com/users/tacaswell/receive...,User,False,0.0,573.0,closed,2018-02-12T03:21:01Z,2018-11-27T00:38:13Z,2018-03-02T08:00:00Z,2018-03-05T05:24:44Z
3,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://github.com/matplotlib/matplotlib/issue...,255766137,MDU6SXNzdWUyNTU3NjYxMzc=,9163,Legends cut off in Jupyter Notebooks when plot...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://github.com/matplotlib/matplotlib/issue...,255594717,MDU6SXNzdWUyNTU1OTQ3MTc=,9162,Using matplotlib 2.1.0rc1 seems to corrupt PySide,...,https://api.github.com/users/tacaswell/receive...,User,False,0.0,1321.0,closed,2014-01-13T03:21:09Z,2019-06-01T18:52:12Z,None,2017-10-03T16:31:17Z
5,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://github.com/matplotlib/matplotlib/issue...,255526966,MDU6SXNzdWUyNTU1MjY5NjY=,9161,image cropped with imsave to SVG on RGB array,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://github.com/matplotlib/matplotlib/issue...,255406393,MDU6SXNzdWUyNTU0MDYzOTM=,9160,blank space vs. white space,...,https://api.github.com/users/tacaswell/receive...,User,False,0.0,573.0,closed,2018-02-12T03:21:01Z,2018-11-27T00:38:13Z,2018-03-02T08:00:00Z,2018-03-05T05:24:44Z
7,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://api.github.com/repos/matplotlib/matplo...,https://github.com/matplotlib/matplotlib/issue...,255380772,M

In [56]:
idf = pd.concat(issues_all, sort="False")

In [57]:
idf.iloc[-1]

assignee                                                                NaN
assignee.avatar_url                                                     NaN
assignee.events_url                                                     NaN
assignee.followers_url                                                  NaN
assignee.following_url                                                  NaN
                                                ...                        
user.site_admin                                                       False
user.starred_url          https://api.github.com/users/dstansby/starred{...
user.subscriptions_url    https://api.github.com/users/dstansby/subscrip...
user.type                                                              User
user.url                              https://api.github.com/users/dstansby
Name: 99, Length: 95, dtype: object

In [59]:
idf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5900 entries, 0 to 99
Data columns (total 95 columns):
assignee                                 0 non-null object
assignee.avatar_url                      116 non-null object
assignee.events_url                      116 non-null object
assignee.followers_url                   116 non-null object
assignee.following_url                   116 non-null object
assignee.gists_url                       116 non-null object
assignee.gravatar_id                     116 non-null object
assignee.html_url                        116 non-null object
assignee.id                              116 non-null float64
assignee.login                           116 non-null object
assignee.node_id                         116 non-null object
assignee.organizations_url               116 non-null object
assignee.received_events_url             116 non-null object
assignee.repos_url                       116 non-null object
assignee.site_admin                      1

In [60]:
idf.to_csv("issues.csv", index=False)